<a href="https://colab.research.google.com/github/geun-hyoung/Data_Analysis/blob/main/Linear_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import statsmodels.api as sm   ### statsmodels.api는 선형모델을 만들기 위한 라이브러리 ###
from sklearn.preprocessing import scale
import pandas as pd
from scipy.stats import probplot
import matplotlib.pyplot as plt
from google.colab import drive    ### colab에서 데이터를 불러오기 위한 코드 ###
drive.mount('/content/drive')        ### colab에서 데이터를 불러오는 라이브러리: drive   /   데이터가 저장된 폴더:  /content/drive'  ###

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/WinterVacation/bostonhousing.csv'      ### ToyotaCorolla.csv라는 데이터가 저장된 경로를 filename에 저장 ###
data = pd.read_csv(filename)     ### filename에 저장된 ToyotaCorolla.csv를 data1에 저장 ###
data.head()        

In [ ]:
col_name = data.columns.tolist()
print(col_name)

In [ ]:
print(data.shape) 

In [ ]:
print(np.arange(0, 10))

In [ ]:
data_regression = np.array(data)
print(np.array(data_regression[:,np.arange(0,13)]))

In [ ]:
# Csv > DataFrame(pandas) > array - statasmodel 함수의 사용 가능
data_regression = np.array(data)
X = np.array(data_regression[:,np.arange(0,13)]) # list indexing에서 콤마는 행과 열을 구분
y = np.array(data_regression[:,13])

In [ ]:
X1 = scale(X)

In [ ]:
print(X1)

In [ ]:
print(y)

In [ ]:
regression_X = sm.add_constant(X1, prepend=False)      # y절편을 찾기 위해 첫번째 행은 모두 1로 채워진 행으로 구성해야 함 ###
reg_model = sm.OLS(y, regression_X)     # OLS = 최소자승법
model = reg_model.fit()

#### 데이터 분석 시 X의 계수만 보면 안됨 -> 값의 척도가 다르기 때문이다. -> 따라서 스케일링이 필요하다.

In [ ]:
print(model.summary())

In [ ]:
## 선형 모델이 잘 만들었는지를 검증하기 위해 실제 값과 예측값이 얼마나 일치하는지를 확인해봐야 함 #
predicted_value = model.predict()   ## res.predict()는 예측값을 도출하는 함수임 #

##  실제 값과 예측값이 일치하는 정도를 시각적으로 확인 #
import matplotlib.pylab as pylab

z = np.polyfit(y, predicted_value, 1)
p = np.poly1d(z)

pylab.plot(y, predicted_value, 'o')      ## x축에는 실제 y값, y축에는 예측된 y값으로  설정하고, x와 y를 비교함 -> 점들이 최대한 일직선 (y=x 선) 위에 있을 수록 예측이 잘되었다고 판단할 수 있음#
pylab.plot(predicted_value,p(predicted_value),"r--")
print( "y=%.6fx+(%.6f)"%(z[0],z[1]))

In [ ]:
# 잔차 (residuals) 란 실제 y값과 예측된 y값 간의 차이를 의미함 #
# 선형모델의 중요한 가정: 잔차는 정규분포를 따른다! #
residuals = model.resid

plt.figure(figsize=(5,5))
probplot(residuals,  dist="norm", plot=plt)   # probplot이라는 함수는 잔차가 정규분포를 따르는지 확인하기 위한 함수 -> 점들이 붉은색 직선위에 있을 수록 잔차는 정규분포를 따를 가능성이 높다고 판단함 #
plt.show()

#### 이상치 제거

In [ ]:
 #일반적으로 잔차가 클 수록 정규분포에서 벗어나게 됨 -> 잔차가 크기가 상대적으로 큰 경우를 제거하고 선형모델을 다시 만드는 것이 좋음 #
regression_X1 = regression_X[abs(residuals) < 5,]      # 잔차 (residuals에 저장됨)가 -3000보다 작거나, 3000보다 큰 경우를 제거하고 데이터를 다시 구성 -> 잔차가 크기가 상대적으로 큰 경우를 제거 #
regression_y1 = y[abs(residuals) < 5,]                 # 잔차 (residuals에 저장됨)가 -3000보다 작거나, 3000보다 큰 경우를 제거하고 데이터를 다시 구성 -> 잔차가 크기가 상대적으로 큰 경우를 제거 #

#regression_X1 = regression_X
#regression_y1 = np.log(y)

In [ ]:
reg_model1 = sm.OLS(regression_y1, regression_X1)
model1 = reg_model1.fit()
print(model1.summary())

In [ ]:
predicted_value = model1.predict()

##  실제 값과 예측값이 일치하는 정도를 시각적으로 확인 #
z = np.polyfit(regression_y1, predicted_value, 1)
p = np.poly1d(z)

pylab.plot(regression_y1, predicted_value, 'o')          ## x축에는 실제 y값, y축에는 예측된 y값으로  설정하고, x와 y를 비교함 -> 점들이 최대한 일직선 (y=x 선) 위에 있을 수록 예측이 잘되었다고 판단할 수 있음#
pylab.plot(predicted_value,p(predicted_value),"r--")

print( "y=%.6fx+(%.6f)"%(z[0],z[1]))

In [ ]:
residuals = model1.resid

plt.figure(figsize=(5,5))
probplot(residuals,  dist="norm", plot=plt)
plt.show()